<a href="https://colab.research.google.com/github/shubhamarora03/Deep_Learning/blob/master/EntityEmbedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

In [0]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model

In [0]:
train=pd.read_csv("./drive/My Drive/Data/train.csv")
test =pd.read_csv("./drive/My Drive/Data/test.csv")

In [4]:
train.target.unique()

array([0, 1])

In [5]:
train.head()

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,nom_5,nom_6,nom_7,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
0,0,0.0,0.0,0.0,F,N,Red,Trapezoid,Hamster,Russia,Bassoon,de4c57ee2,a64bc7ddf,598080a91,0256c7a4b,02e7c8990,3.0,Contributor,Hot,c,U,Pw,6.0,3.0,0
1,1,1.0,1.0,0.0,F,Y,Red,Star,Axolotl,NaN,Theremin,2bb3c3e5c,3a3a936e8,1dddb8473,52ead350c,f37df64af,3.0,Grandmaster,Warm,e,X,pE,7.0,7.0,0
2,2,0.0,1.0,0.0,F,N,Red,NaN,Hamster,Canada,Bassoon,b574c9841,708248125,5ddc9a726,745b909d1,NaN,3.0,NaN,Freezing,n,P,eN,5.0,9.0,0
3,3,NaN,0.0,0.0,F,N,Red,Circle,Hamster,Finland,Theremin,673bdf1f6,23edb8da3,3a33ef960,bdaa56dd1,f9d456e57,1.0,Novice,Lava Hot,a,C,NaN,3.0,3.0,0
4,4,0.0,NaN,0.0,T,N,Red,Triangle,Hamster,Costa Rica,NaN,777d1ac2c,3a7975e46,bc9cc2a94,NaN,c5361037c,3.0,Grandmaster,Cold,h,C,OZ,5.0,12.0,0


In [0]:
test['target']=-1

In [7]:
test.head()

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,nom_5,nom_6,nom_7,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
0,600000,0.0,0.0,0.0,F,Y,Blue,Polygon,Axolotl,Finland,Piano,52f6dd16c,147d704e4,8d857a0a1,ca9ad1d4b,fced9e114,3.0,Novice,Boiling Hot,f,U,oU,3.0,9.0,-1
1,600001,0.0,0.0,0.0,F,Y,Red,Circle,Lion,Russia,Bassoon,691ebeae8,8653dcc2e,67a8d4ebb,060a21580,7ca8775da,1.0,Novice,Cold,n,N,NaN,2.0,8.0,-1
2,600002,0.0,0.0,0.0,F,Y,Blue,Circle,Axolotl,Russia,Theremin,81f792c16,6cdda499e,69403e18c,165e81a00,5940334c9,1.0,Expert,Warm,i,N,DN,2.0,6.0,-1
3,600003,1.0,0.0,0.0,F,N,Red,Polygon,Axolotl,Costa Rica,Bassoon,c9134205b,acbca4827,cb681246b,77d41330d,6fbdeefc8,1.0,Expert,Hot,m,B,AG,1.0,6.0,-1
4,600004,0.0,0.0,1.0,F,Y,Red,Circle,NaN,Finland,Theremin,f0f100f57,6f800b9af,cd9feb5c6,2218d9dfe,2a27c8fde,1.0,Contributor,Lava Hot,o,J,DT,3.0,3.0,-1


In [8]:
train.shape , test.shape

((600000, 25), (400000, 25))

In [0]:
full_data = pd.concat([train,test]).reset_index(drop=True)

In [10]:
full_data.shape

(1000000, 25)

In [0]:
features=[f for f in train.columns if f not in ["id","target"]]

In [12]:
features

['bin_0',
 'bin_1',
 'bin_2',
 'bin_3',
 'bin_4',
 'nom_0',
 'nom_1',
 'nom_2',
 'nom_3',
 'nom_4',
 'nom_5',
 'nom_6',
 'nom_7',
 'nom_8',
 'nom_9',
 'ord_0',
 'ord_1',
 'ord_2',
 'ord_3',
 'ord_4',
 'ord_5',
 'day',
 'month']

In [0]:
for col in features:
  full_data.loc[:,col]=full_data.loc[:,col].astype(str).fillna("-1")

In [14]:
full_data.target.unique()

array([ 0,  1, -1])

In [0]:
for col in features:
  lbl_enc = preprocessing.LabelEncoder()
  full_data.loc[:,col] = lbl_enc.fit_transform(full_data[col])

In [0]:
full_data.head()

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,nom_5,nom_6,nom_7,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
0,0,0,0,0,0,0,2,4,3,5,0,1059,1013,87,1,27,2,0,3,2,20,56,5,5,0
1,1,1,1,0,0,1,2,3,0,6,3,210,359,27,69,2112,2,2,5,4,23,151,6,9,0
2,2,0,1,0,0,0,2,6,3,0,0,860,694,90,102,2218,2,5,2,13,15,105,4,11,0
3,3,2,0,0,0,0,2,0,3,3,3,477,241,51,170,2167,0,4,4,0,2,140,2,5,0
4,4,0,2,0,1,0,2,5,3,2,4,556,361,182,222,1747,2,2,1,7,2,50,4,3,0


In [0]:
train = full_data[full_data.target!=-1].reset_index(drop=True)
test  = full_data[full_data.target==-1].reset_index(drop=True)

In [17]:
train.shape , test.shape

((600000, 25), (400000, 25))

In [18]:
int(min(np.ceil(1000/2),50))

50

Converting list of list of features

In [0]:
def get_model(df,columns):
  inputs=[]
  outputs=[]
  for c in columns:
    num_uniq_values=int(df[c].nunique())
    emb_dim = int(min(np.ceil(num_uniq_values/2),50))
    #One embedding layer for each variable
    inp = layers.Input(shape=(1,))
    out = layers.Embedding(num_uniq_values+1,emb_dim)(inp)
    out = layers.Reshape(target_shape=(emb_dim,))(out)
    #Appending all the categorical inputs
    inputs.append(inp)
    #Appending all the embeddings
    outputs.append(out)
  x = layers.Concatenate()(outputs)
  x = layers.Dense(300,activation='relu')(x)
  x = layers.Dropout(0.3) (x)
  y = layers.Dense(1,activation='sigmoid')(x)
  model = Model(inputs=inputs,outputs=y)
  return model

In [20]:
get_model(train,features).summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 1)]          0                                            
______________________________________________________________________________________________

In [0]:
model = get_model(train,features)
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])

In [0]:
[train.head(10).loc[:,f].values for f in features]

[array([0, 1, 0, 2, 0, 0, 0, 0, 0, 0]),
 array([0, 1, 1, 0, 2, 2, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 1, 0, 1, 0, 2]),
 array([0, 0, 0, 0, 1, 1, 0, 1, 0, 0]),
 array([0, 1, 0, 0, 0, 0, 0, 0, 0, 1]),
 array([2, 2, 2, 2, 2, 2, 2, 2, 0, 2]),
 array([4, 3, 6, 0, 5, 5, 5, 5, 1, 1]),
 array([3, 0, 3, 3, 3, 4, 3, 0, 3, 3]),
 array([5, 6, 0, 3, 2, 1, 2, 3, 5, 3]),
 array([0, 3, 0, 3, 4, 0, 0, 0, 1, 3]),
 array([1059,  210,  860,  477,  556,  766,  645,   83,    6,  754]),
 array([1013,  359,  694,  241,  361, 1059, 1222,  277, 1003, 1387]),
 array([ 87,  27,  90,  51, 182, 138,  25, 124,  64,  84]),
 array([  1,  69, 102, 170, 222,  93, 167, 169, 138,   2]),
 array([  27, 2112, 2218, 2167, 1747,   59,  692, 1464, 2218, 1086]),
 array([2, 2, 2, 0, 2, 1, 0, 2, 0, 2]),
 array([0, 2, 5, 4, 2, 1, 2, 1, 4, 0]),
 array([3, 5, 2, 4, 1, 3, 1, 1, 0, 4]),
 array([ 2,  4, 13,  0,  7,  1,  2,  1,  2, 13]),
 array([20, 23, 15,  2,  2, 16, 17, 24, 13,  8]),
 array([ 56, 151, 105, 140,  50, 181, 159,  54, 13

In [23]:
model.fit([train.loc[:,f].values for f in features],
          train.target.values,
          batch_size=64,
          epochs=5)

Epoch 1/5
9375/9375 [==============================] - 89s 9ms/step - loss: 0.3925 - accuracy: 0.8293
Epoch 2/5
9375/9375 [==============================] - 88s 9ms/step - loss: 0.3801 - accuracy: 0.8353
Epoch 3/5
9375/9375 [==============================] - 88s 9ms/step - loss: 0.3666 - accuracy: 0.8431
Epoch 4/5
9375/9375 [==============================] - 86s 9ms/step - loss: 0.3522 - accuracy: 0.8500
Epoch 5/5
9375/9375 [==============================] - 86s 9ms/step - loss: 0.3374 - accuracy: 0.8570
